<a href="https://colab.research.google.com/github/UOS-COMP1201/code/blob/main/integerprogramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

In [6]:

c=np.array([4,5])
A=np.array([[1,4],[3,-4]])
#b=np.array([10,6])
#b=np.array([90,60])
#b=np.array([80,70])     
b=np.array([101,70])
x1_bounds=(0,None)
x2_bounds=(0,None)

In [7]:

import copy
max=0

def IP(c,A,b,bounds):
  global max
  res=linprog(c,A,b,bounds=bounds)
  if res.fun == None:
    print("pruned by infeasibility")
    return 
  if -res.fun<max:
    print(-res.fun,res.x)
    print("pruned by bound")
    return 
  result=-res.fun
  # at this point result>max
  print(result,res.x)
  int_sol=True
  for var in res.x:
    if np.floor(var)!=np.ceil(var):
      int_sol=False
      break
  if  int_sol:
        max=result
        return
  # not all variables are integers, branch      
  for i,var in enumerate(res.x):
    fl=np.floor(var)
    cl=np.ceil(var)
    
    if fl!=cl:
      print("               ",fl,cl)
      tmp=copy.deepcopy(bounds)
      tmp[i]=(cl,tmp[i][1])
      #r1=linprog(c,A,b,bounds=bounds)
      IP(c,A,b,tmp)
      #print(r1.fun)
      tmp=copy.deepcopy(bounds)
      tmp[i]=(tmp[i][0],fl)
      #r2=linprog(c,A,b,bounds=bounds)

      IP(c,A,b,tmp)
      #print(r2.fun)
   

  return res.fun,res.x

In [8]:

c=[-x for x in c]
r=IP(c,A,b,[x1_bounds,x2_bounds])


243.8125 [42.75   14.5625]
                42.0 43.0
pruned by infeasibility
241.75 [42.   14.75]
                14.0 15.0
239.0 [41. 15.]
238.0 [42. 14.]
pruned by bound
                14.0 15.0
239.0 [41. 15.]
238.0 [42. 14.]
pruned by bound
